## Add SB1 geographic information.

In [ ]:
import _utils
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp.sql import to_snakecase

In [ ]:
import fuzzywuzzy
from fuzzywuzzy import process

In [ ]:
import fsspec
from calitp import *
from calitp.storage import get_fs

fs = get_fs()
import os

In [ ]:
pd.options.display.max_columns = 200
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
def basic_cleaning(df, agency_col: str,
                   project_name_col:str,
                   project_id_col: str, 
                   project_desc_col:str,
                   data:str):
    
    df = _utils.organization_cleaning(df, agency_col)
    
    # Remove all punctation, lowercase, and strip whitespaces from 
    # project titles & descriptions. Count number of strings.
    for i in [project_name_col, project_desc_col]:
        df[i] = (df[i].str.lower().str.replace('[^\w\s]','').str.strip())
        df[f"{i}_count"] = df[i].str.count('\w+')
                 
    # Some project names contain the year. Remove anything after 20..
    df[project_name_col] = df[project_name_col].str.split("20").str[0]
    
    # Project ID, remove all commas and lowercase if there are strings
    df[project_id_col] = (df[project_id_col].str.replace("'", "").str.lower().str.strip())
    
    # Get rid of | in object cols
    # https://stackoverflow.com/questions/68152902/extracting-only-object-type-columns-in-a-separate-list-from-a-data-frame-in-pand
    string_cols = df.select_dtypes(include=['object']).columns.to_list()
    try:
        for i in string_cols:
            df[i] = df[i].str.replace("|", "")
    except:
        pass
    
    # Try to extract titles from popups
    try:
        df["popup"] = df['popup'].str.split("<br  />").str[1].str.split("20").str[0].str.lower().str.strip().str.replace('[^\w\s]','')
    except:
        pass
 
    return df

### Non SHOPP
* No year information for projects.

In [ ]:
# Read in 10 Year non SHOPP with ATP and TIRCP
nonshopp = to_snakecase(
    pd.read_excel(f"{_utils.GCS_FILE_PATH}cleaned_data_atp_tircp.xlsx")
)

In [ ]:
# Subset to join.
non_shopp_subset = [
    "ppno",
    "ct_project_id",
    "ea",
    "project_name",
    "lead_agency",
    "previous_caltrans_nominations",
    "full_county_name",
    "county",
    "district_full_name",
    "district",
    "project_description",
    "current_phase",
    "primary_mode",
    "urban_rural",
    "total_project_cost__$1,000",
    "total_unfunded_need__$1,000",
    "shs_capacity_increase_detail",
    "current_phase",
]

In [ ]:
# nonshopp = nonshopp[non_shopp_subset]

In [ ]:
# Add a digit in front of single digits
nonshopp.district = nonshopp.district.map("{:02}".format)

In [ ]:
nonshopp = basic_cleaning(nonshopp, "lead_agency", "project_name", "ct_project_id", "project_description", "nonshopp")

In [ ]:
nonshopp[['project_name', 'project_name_count']].sample(3)

### 9 Sample Non SHOPP 

In [ ]:
nine_projects_names = [
    "LA-210 Median Concrete Barrier Renovation",
    "SR-14 Widening Project",
    "US 395 Freight Mobility and Safety Project",
    "East Bay Greenway Multimodal Corridor Project",
    "Watsonville-Santa Cruz Multimodal Corridor Program",
    "SM 101 Woodside Road Interchange and Port Access Project",
    "I-710 Integrated Corridor Management",
    "Five Cities Multimodal Transportation Network Enhancement Project",
    "SR-86/Avenue 50 New Interchange (Phase II)",
]

In [ ]:
# nine_projects_names = [x.lower() for x in nine_projects_names]

In [ ]:
nine_projects_id = [
    "0422000202",
    "0414000032",
    "0520000083",
    "0515000063",
    "0721000056",
    "0716000370",
    "0813000222",
    "0814000144",
    "0414000032",
    "0720000165",
]

In [ ]:
# nine_sample_projects = (nonshopp[nonshopp.ct_project_id.isin(nine_projects_id)].reset_index(drop=True))

* Solutions for Congest Corridors (SCCP): 1
* Trade Corridor Enhancement Program (TCEP): 3
* Only 3 projects seem to have been awarded. 
    * east bay greenway multimodal corridor project phase 1
    * us 101woodside road interchange and port access project
    * watsonvillesanta cruz multimodal corridor program wscmcp cycle 3 project contract 1 sr 1 freedom to state park aux lanes bus on shoulders and coastal rail trail segment 12

In [ ]:
# nine_sample_projects[['project_name','project_description','county','previous_caltrans_nominations']]

### Sb1 Geo
* https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_032022/FeatureServer

In [ ]:
# Subset to preview SB1 vs Nonshopp. Nonshopp is on the left, sb1 on the right
preview_cols =  ['project_name','projecttitle','project_description','projectdescription', 'full_county_name', 'countynames']

In [ ]:
url_pt1 = "https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_032022/FeatureServer/"
url_pt2 = "/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*+&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=&resultOffset=&resultRecordCount=&returnTrueCurves=false&sqlFormat=none&f=geojson"

In [ ]:
def rest_server():
    full_gdf = pd.DataFrame()
    for i in [*range(0,22)]:
        df = to_snakecase(gpd.read_file(f"{url_pt1}{i}{url_pt2}"))
        full_gdf = pd.concat([full_gdf, df], axis=0)
    return full_gdf

In [ ]:
sb1_geo1 = rest_server()

In [ ]:
# tircp = to_snakecase(gpd.read_file("https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_032022/FeatureServer/5/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*+&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=&resultOffset=&resultRecordCount=&returnTrueCurves=false&sqlFormat=none&f=geojson"))

In [ ]:
atp = to_snakecase(gpd.read_file("https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_032022/FeatureServer/12/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*+&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=&resultOffset=&resultRecordCount=&returnTrueCurves=false&sqlFormat=none&f=geojson"))

In [ ]:
atp.sample()

In [ ]:
sb1_geo2 = basic_cleaning(sb1_geo1, 'agencies','projecttitle','projectid',
                         'projectdescription', 'sb1')

In [ ]:
sb1_geo2['projecttitle'] = sb1_geo2['projecttitle'].fillna(sb1_geo2['popup'])

In [ ]:
sb1_geo2.programcodes.value_counts()

#### Check geometries
* Take away invalid row.

In [ ]:
# All geometry is valid
#sb1_geo2.geometry.is_valid.sum() == len(sb1_geo2)

In [ ]:
#sb1_geo2.geometry.notna().sum()  == len(sb1_geo2)

In [ ]:
#len(sb1_geo2[~sb1_geo2.geometry.is_empty]) 

In [ ]:
# Throw out missing geometry
missing_geo = sb1_geo2[sb1_geo2.geometry.is_empty]

In [ ]:
len(missing_geo)

In [ ]:
sb1_geo2 = sb1_geo2[~sb1_geo2.geometry.is_empty].reset_index(drop = True)

In [ ]:
# len(sb1_geo2) == len(sb1_geo2[~sb1_geo2.geometry.is_empty]) 

In [ ]:
sb1_geo2.shape

In [ ]:
# sb1_geo.countynames.sort_values().unique()

In [ ]:
# sb1_geo.explore()

### Read in all projects
* Need this because not every file has project names.

In [ ]:
sb1_all_projects_url = f"{url_pt1}22{url_pt2}"

In [ ]:
sb1_all_projects = to_snakecase(gpd.read_file(sb1_all_projects_url))

In [ ]:
sb1_all_projects.geometry.value_counts()

In [ ]:
# No geometry, just drop it
sb1_all_projects = sb1_all_projects.drop(columns=["geometry"])

In [ ]:
sb1_all_projects = basic_cleaning(sb1_all_projects, 'implementingagency','projecttitle','projectid',
                         'projectdescription', 'sb1')

In [ ]:
sb1_all_projects.shape, sb1_all_projects.projecttitle.nunique()

In [ ]:
# Project ID matches...
all_projects_project_id = set(sb1_all_projects.projectid.unique().tolist())
sb1_geo2_project_id = set(sb1_geo2.projectid.unique().tolist())

In [ ]:
len(list(set(sb1_geo2_project_id).intersection(all_projects_project_id)))

In [ ]:
len(sb1_geo2_project_id - all_projects_project_id)

In [ ]:
len(sb1_geo2)

In [ ]:
# None of the object id matches 
all_projects_object_id = set(sb1_all_projects.objectid.unique().tolist())
sb1_geo2_object_id = set(sb1_geo2.objectid.unique().tolist())

In [ ]:
len(sb1_geo2_object_id - all_projects_object_id), len(sb1_geo2_object_id)

In [ ]:
len(all_projects_object_id - sb1_geo2_object_id)

In [ ]:
# set(sb1_all_projects.columns).difference(set(sb1_csv.columns))

In [ ]:
m1 = pd.merge(
    sb1_geo2,
    sb1_all_projects[['projectid','projecttitle']],
    how="left",
    on=["projectid"],
)

In [ ]:
m1.projecttitle_x.value_counts().head()

In [ ]:
m1.projecttitle_y.isna().sum()

In [ ]:
sb1_all_projects.columns, sb1_geo2.columns

In [ ]:
len(sb1_all_projects), len(sb1_geo2), 5513+4415

In [ ]:
pd.merge(
    sb1_all_projects,
    sb1_geo2,
    how="left",
    left_on=[ "ct_districts", "agencyid", "totalcost", "fiscalyearcode"],
    right_on=["ct_districts", "agencyids", "totalcost", "fiscalyearcodes"],
    indicator=True,
)[["_merge"]].value_counts()

In [ ]:
m1 = pd.merge(
    sb1_all_projects,
    sb1_geo2[['projecttitle', "programcodes", "totalcost", "agencies", "countynames"]],
    how="right",
    left_on=["programcodes", "totalcost", "implementingagency", "countynames"],
    right_on=["programcodes", "totalcost", "agencies", "countynames"],
    indicator=True,
)

In [ ]:
m1.projecttitle_x = m1.projecttitle_x.fillna(m1.projecttitle_y)

In [ ]:
m1._merge.value_counts(), len(m1)

In [ ]:
m1[['projecttitle_x','programcodes',]].sort_values('programcodes').head()

In [ ]:
m3 = pd.merge(
    sb1_geo2,
    sb1_all_projects[['projecttitle', "programcodes", "totalcost", "implementingagency", "fiscalyearcode"]],
    how="left",
    left_on=[ "programcodes", "totalcost", "agencies","fiscalyearcodes"],
    right_on=[ "programcodes", "totalcost", "implementingagency","fiscalyearcode"],
    indicator = True
)

In [ ]:
m3._merge.value_counts()

In [ ]:
m3.projecttitle_x = m3.projecttitle_x.fillna(m3.projecttitle_y)

In [ ]:
m3.projecttitle_x = m3.projecttitle_x.fillna('None')

In [ ]:
m3[['projecttitle_x','programcodes',]].sort_values('programcodes').head()

In [ ]:
# m2[['projecttitle_x','projecttitle_y', 'programcodes',]].sort_values('programcodes')

In [ ]:
m4 = pd.merge(
    sb1_geo2,
    sb1_all_projects[['projecttitle', "programcodes", "totalcost", "implementingagency", "ct_districts", "fiscalyearcode"]],
    how="left",
    left_on=[ "programcodes", "totalcost", "agencies", "ct_districts", "fiscalyearcodes"],
    right_on=[ "programcodes", "totalcost", "implementingagency", "ct_districts",  "fiscalyearcode",],
    indicator=True,
)

In [ ]:
m4._merge.value_counts()

In [ ]:
type(m4), m4.shape

In [ ]:
m4.projecttitle_x = m4.projecttitle_x.fillna(m4.projecttitle_y)

In [ ]:
m4.projecttitle_x = m4.projecttitle_x.fillna('None')

### Compare with ALL Projects

In [ ]:
nonshopp_sb1_m1 = pd.merge(
    m3.loc[m3.projecttitle_x != "None"].drop(columns = ["_merge"])[['projecttitle_x','projectdescription','countynames','geometry']],
    nonshopp,
    how="inner",
    left_on=[ "projecttitle_x"],
    right_on=["project_name",],
    indicator=True,
)

In [ ]:
len(nonshopp_sb1_m1), type(nonshopp_sb1_m1)

In [ ]:
# nonshopp_sb1_m1.explore()

In [ ]:
# nonshopp_sb1_m1[['previous_caltrans_nominations','project_name','projecttitle_x','project_description','projectdescription', 'full_county_name', 'countynames']]

In [ ]:
found_projects = nonshopp_sb1_m1.projecttitle_x.unique().tolist()

In [ ]:
# Filter out results before fuzzy matching
m3 = m3[~m3["projecttitle_x"].isin(found_projects)].reset_index(drop = True)

In [ ]:
# Filter out the projects already found above.
nonshopp2 = nonshopp[~nonshopp["project_name"].isin(found_projects)].reset_index(drop = True)

#### Fuzzy Matches

##### Try with Project Titles

In [ ]:
# Replace all rows in agency column with a min ratio with  "string_to_match value"
def replace_matches_in_column(df, column, new_col_name, string_to_match, min_ratio):
    # Get a list of unique strings
    strings = df[column].unique()

    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(
        string_to_match, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio
    )

    # Only get matches with a  min ratio
    close_matches = [matches[0] for matches in matches if matches[1] > min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches
    df.loc[rows_with_matches, new_col_name] = string_to_match

In [ ]:
nonshopp_projects = nonshopp2.project_name.unique().tolist()

In [ ]:
m3["projecttitle_x_count"] = m3["projecttitle_x"].str.count('\w+')

In [ ]:
# Delete project titles that are short
sb1_w_projectnames = (m3.loc[m3.projecttitle_x_count > 3]).reset_index(drop = True)

In [ ]:
sb1_w_projectnames.shape

In [ ]:
for i in nonshopp_projects:
    replace_matches_in_column(
        sb1_w_projectnames
        , "projecttitle_x", "project_title_fuzzy_match", i,90 
    )

In [ ]:
# Drop nulls and duplicates
fuzzy_match_results = (sb1_w_projectnames.loc[sb1_w_projectnames.project_title_fuzzy_match.notnull()]
                       .drop_duplicates(subset = ["projecttitle_x", "project_title_fuzzy_match", "projectdescription"])
                       .reset_index(drop = True)
                       .sort_values('projecttitle_x')
                      )

In [ ]:
len(fuzzy_match_results)

In [ ]:
# Last one isn't the same.
fuzzy_match_results[['projecttitle_x','project_title_fuzzy_match', 'countynames', 'fiscalyears']]

In [ ]:
nonshopp_sb1_m2 = pd.merge(
    fuzzy_match_results.drop(columns = ["_merge"]).head(4)[['projecttitle_x','project_title_fuzzy_match','projectdescription','countynames','geometry']],
    nonshopp2,
    how="inner",
    left_on=["project_title_fuzzy_match", "countynames"],
    right_on=["project_name", "full_county_name"],
)

In [ ]:
type(nonshopp_sb1_m2), len(nonshopp_sb1_m2)

In [ ]:
found_projects2 = nonshopp_sb1_m2.projecttitle_x.unique().tolist()

In [ ]:
found_projects3 = nonshopp_sb1_m2.project_title_fuzzy_match.unique().tolist()

In [ ]:
m3 = m3[~m3["projecttitle_x"].isin(found_projects2)].reset_index(drop = True)

In [ ]:
# Filter out the projects already found above.
nonshopp2 = nonshopp2[~nonshopp2["project_name"].isin(found_projects3)].reset_index(drop = True)

##### Try with project description since titles are very vague.

In [ ]:
asdfsadf

In [ ]:
def replace_matches_set_ratio(df, column, new_col_name, string_to_match, min_ratio):
    # Get a list of unique strings
    strings = df[column].unique()

    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(
        string_to_match, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_set_ratio
    )

    # Only get matches with a  min ratio
    close_matches = [matches[0] for matches in matches if matches[1] > min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches
    df.loc[rows_with_matches, new_col_name] = string_to_match

In [ ]:
# Only include projects with a long enough desc
# Drop dups
sb1_w_projectdesc = ((sb1_geo2.loc[sb1_geo2.projectdescription_count > 10])
.drop_duplicates(subset = ["projecttitle", "projectdescription", "countynames"])
.reset_index(drop = True)
                    )

In [ ]:
# Test with nonshopp 
nonshopp_with_desc = ((nonshopp.loc[nonshopp.project_description_count > 10])
                     .drop_duplicates(subset = ["project_name", "project_description", "full_county_name"]).reset_index(drop = True)
                    )

In [ ]:
# Reverse -> replace 
for i in sb1_w_projectdesc["projectdescription"].loc[3000:3515].unique().tolist():
    replace_matches_set_ratio(
        nonshopp_with_desc, "project_description", "project_desc_fuzzy_match", i, 95
   )

In [ ]:
nonshopp_with_desc.columns


In [ ]:
nonshopp_with_desc = nonshopp_with_desc.loc[nonshopp_with_desc.project_desc_fuzzy_match.notnull()].reset_index(drop = True)

In [ ]:
fuzzy_descriptions_list = ['in placer county from highway 65 to rocklin road the project will add an auxiliary lane between highway 65 and the rocklin road interchanges providing improved travel time reliability for the more than 90 bus trips that currently pass through this area daily',
 'in sacramento county on watt avenue from i80 westbound ramps to roseville rd  between orange grove avenue and roseville rd construct buffered bike lanes separated pedestrianfriendly sidewalks landscaped medians improved transit facilities for pedestrians including bus turnouts improve street lighting improve signalized intersections and other streetscape amenities to encourage mobility by active modes of transportation and provide community identity  between orange grove avenue to i80 westbound ramps extend class 2 bike lane and sidewalk improvements',
 'from state route 84 alameda county to alcosta boulevard contra costa county project will add one high occupancy hovexpress lane and construct other improvements including replacementupgrade of median concrete barrier and lighting on interstate 680 in the southbound direction between state route sr84 and alcosta boulevard through alameda and contra costa counties',
 'on route 101 in marin county in and near city of novato from just south of the franklin avenue overhead to 03 miles south of the marinsonoma county line  the project will widen route 101 to construct a southbound hov lane from 03 miles south of the marinsonoma county line to just south of the franklin avenue overhead 60 miles and a northbound hov lane from 17 miles north of the atherton avenue overcrossing to 03 miles south of the marinsonoma county line 35 miles the project includes roadway and bridge widening for hov lanes and standard shoulders the project will also upgrade the horizontal and vertical roadway alignment for a 70 mph design speed modify the redwood landfill interchange ramps to conform with the new alignment and restripe a frontage road redwood boulevard for class ii bike lanes in novato',
 'near capitola and aptos state route 1 from state park drive to bayporter interchanges  construct auxiliary lanes between interchanges  includes reconstruction of the capitola avenue overcrossing to accommodate new lanes on state route 1 and improve bicycle and pedestrian facilities hybrid busonshoulderauxiliary lane facility between bay aveporter st and state park dr total distance 3 miles bicyclepedestrian overcrossing of hwy 1 at mar vista dr with sidewalk ada ramps and intersection improvements at bridge approaches additional project elements add emergency pullouts and enforcement areas sound wall retaining walls improved median barrier lighting overhead signs traffic monitoring stations drainage and droughttolerant landscapingsccplpp title watsonvillesanta cruz  multimodal improvements bay aveporter st to state park dr  contract 2',
 'in monterey county at castroville boulevard from post mile r16 to 14  build a new interchange at castroville boulevard and highway 156',
 'in tulare county near the city of tulare at commercial avenue and state route 99 between 09 mile north of avenue 200 oc and paige avenue oc construct new interchange and construct north and south bound auxiliary lanes']

In [ ]:
nonshopp_sb1_m3 = pd.merge(
    m3,
    nonshopp2[nonshopp2.project_description.isin(fuzzy_descriptions_list)],
    how="inner",
    left_on=["projectdescription", "countynames"],
    right_on=["project_description", "full_county_name"],)

In [ ]:
len(nonshopp_sb1_m3)

In [ ]:
# nonshopp_sb1_m3

### Compare with 9 Sample Projects

In [ ]:
# Subset sb1_geo to only programs these 9 projects have applied for
tcep_sccp1 = sb1_geo2[sb1_geo2["programcodes"].str.contains(('TCEP|SCCP'))].reset_index(drop = True)

In [ ]:
# Subset sb1_geo to only programs these 9 projects are located in
tcep_sccp2 = sb1_geo2[sb1_geo2["countynames"].str.contains(('Alameda|San Mateo|Santa Cruz|San Luis Obispo|Los Angeles|San Bernardino|Riverside'))].reset_index(drop = True)

In [ ]:
# Subset sb1_geo to only programs these 9 projects have applied for
tcep_sccp2 = tcep_sccp2[tcep_sccp2["programcodes"].str.contains(('TCEP|SCCP'))].reset_index(drop = True)

In [ ]:
tcep_sccp2.shape

In [ ]:
tcep_sccp2.programcodes.value_counts()

In [ ]:
pd.merge(
    nine_sample_projects,
    tcep_sccp2,
    how="outer",
    left_on=["ct_project_id"],
    right_on=["projectid"],
    indicator=True,
)[["_merge"]].value_counts()

In [ ]:
pd.merge(
    nine_sample_projects,
    tcep_sccp2,
    how="outer",
    left_on=["project_name"],
    right_on=["projecttitle"],
    indicator=True,
)[["_merge"]].value_counts()

* Eyeballing matches
    * route 395 widening from sr 18 to chamberlaine way in SB1 could match us 395 freight mobility and safety project in Non SHOPP
    * state route 1 state park to bayporter auxiliary lanes in SB1 is watsonvillesanta cruz multimodal corridor program wscmcp cycle 3 project contract 1 sr 1 freedom to state park aux lanes bus on shoulders and coastal rail trail segment 12 in non SHOPP

In [ ]:
# tcep_sccp2[['projecttitle','agencies','countynames', 'projectdescription']].sort_values(['countynames','projecttitle'])

In [ ]:
# nine_sample_projects[['project_name','lead_agency','full_county_name','project_description']].sort_values(['full_county_name','project_name'])

In [ ]:
non_shopp_projects_sb1_list = ['route 395 widening from sr 18 to chamberlaine way', 
                             'state route 1  state park to bayporter auxiliary lanes']

In [ ]:
non_shopp_projects_in_sb1 = tcep_sccp2[tcep_sccp2["projecttitle"].isin(non_shopp_projects_sb1_list)].reset_index(drop = True)

In [ ]:
non_shopp_projects_in_sb1.projecttitle = non_shopp_projects_in_sb1.projecttitle.replace({
    'route 395 widening from sr 18 to chamberlaine way': 'us 395 freight mobility and safety project',
    'state route 1  state park to bayporter auxiliary lanes': 'watsonvillesanta cruz multimodal corridor program wscmcp cycle 3 project contract 1  sr 1 freedom to state park aux lanes bus on shoulders and coastal rail trail segment 12'
})

In [ ]:
non_shopp_projects_in_sb1[['agencies','programcodes', 'countynames','projecttitle','projectdescription']]

In [ ]:
len(non_shopp_projects_in_sb1)

In [ ]:
nine_sample_projects_geo = pd.merge(
    non_shopp_projects_in_sb1[['projecttitle','geometry', 'projectdescription']],
    nine_sample_projects,
    how="outer",
    left_on=["projecttitle"],
    right_on=["project_name"],
)

In [ ]:
len(nine_sample_projects_geo)

In [ ]:
type(nine_sample_projects_geo)

In [ ]:
# nine_sample_projects_geo[['project_name','projecttitle','project_description','projectdescription', 'full_county_name']]

In [ ]:
#  _utils.geojson_gcs_export(nine_sample_projects_geo,_utils.GCS_FILE_PATH, 'nine_sample_projects_geom')